<b>Text summarization project<b>

Summarization can be defined as a task of producing a concise and fluent summary while preserving key information and overall meaning.

Text summarization mean producing a concise and fluent summary without deleting the key information and overall meaning.

The summarization methods can be applied in the business such as summarize finanical news to find the impact headline and information, as well as in the scientific such as summarizing research paper information.

In [14]:
!pip install scipy>=1.8

In [2]:
#import useful variable
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [4]:
# Define the function to read the article
# Here note that we will read the sentence in the from of txt file 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines() # read everyline of the data 
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

In [5]:
# We need to build the sentence similarlity matrix
# changing each sentence to the represent vector 
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
# score the similarlity score in each matrix
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


In [8]:
def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    #split the sumart 
    sentences =  read_article(file_name)

    # generate similarity matrix 
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # rank sentence based on it similarity 
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    #pick only the top sentence 
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    #output the summary text
    print("Summarize Text: \n", ". ".join(summarize_text))



Note for the further improvement of the project, we can optimize how we select the number of the sentence to summarize.
